In [2]:
import urllib
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import matplotlib.pyplot as plt
import numpy as np

In [49]:
#SCRAPER FUNCTIONS

def raw_scrape(link):
    driver.get(link)
    driver.find_element_by_xpath('//*[@id="2-400-tab-2"]').click()
    output = list()
    game = driver.find_element_by_xpath('//*[@id="mbt-v2-game-full"]/div[1]').text
    n = 0
    nums = list()
    for letter in game:
        if letter.isdigit() == True:
            nums.append(n)
        n = n+1
    home = game[0:nums[0]-1]
    away = game[nums[-1]+2:]
    output = list()
    output.append([home,away,"",""])
    j = 0

    time.sleep(2)
    while True:
        entry = list()
        j = j + 1
        try:
            times = driver.find_element_by_xpath('//*[@id="2-400-play-by-play-actions-container"]/table/tbody/tr['+str(j)+']/td[1]').text
            action = driver.find_element_by_xpath('//*[@id="2-400-play-by-play-actions-container"]/table/tbody/tr['+str(j)+']/td[4]').text
            score = driver.find_element_by_xpath('//*[@id="2-400-play-by-play-actions-container"]/table/tbody/tr['+str(j)+']/td[2]').text
            try:
                team = driver.find_element_by_xpath('//*[@id="2-400-play-by-play-actions-container"]/table/tbody/tr['+str(j)+']/td[3]/img').get_attribute("alt")
            except:
                pass
            output.append([times,team,action,score])
            
        except:
            try: 
                action = driver.find_element_by_xpath('//*[@id="2-400-play-by-play-actions-container"]/table/tbody/tr['+str(j)+']/td/b').text
                output.append([times,team,action,score])
                #print("tried")
                #print(j)
            except:
                try:
                    action = driver.find_element_by_xpath('//*[@id="2-400-play-by-play-actions-container"]/table/thead/tr/th['+str(j)+']')
                    #print("except then tried")
                except:
                    try:
                        action = driver.find_element_by_xpath('//*[@id="2-400-play-by-play-actions-container"]/table/tbody/tr['+str(j)+']/td').text
                    except:    
                        #print("except")
                        #print(j)      
                        break
    return output



def raw_v1_topbp(output):
    i=1
    newrow = ["00:00", "Pregame",0,0,0,"","","","","","","","","",""]
    pbp_fin = list()
    pbp_fin.append(newrow.copy())
    for i in range(1,len(output)):
        newrow[0] = output[i][0]
        newrow[1] = output[i][2]
        if output[i][3] != "":
            score = output[i][3]
            colon_ind = score.find(":")
            newrow[2] = int(score[0:colon_ind])
            newrow[3] = int(score[colon_ind+1:])
            newrow[4] = newrow[2] - newrow[3]
        if "left" in output[i][2]:
            sub = output[i][2]
            marker = sub.find("left")
            subout = sub[0:marker]
        
            if output[i][1] == output[0][0]:
                for m in range(5,10):
                    if newrow[m] == subout:
                        newrow[m] = ""
                        break
            else:
                for m in range(10,15):
                    if newrow[m] == subout:
                        newrow[m] = ""
                        break
                    
    
        if "entered" in output[i][2]:
            sub = output[i][2]
            marker = sub.find("entered")
            subin = sub[0:marker]
            if output[i][1] == output[0][0]:
                for m in range(5,10):
                    if newrow[m] == "":
                        newrow[m] = subin
                        break
            else:
                for m in range(10,15):
                    if newrow[m] == "":
                        newrow[m] = subin 
                        break
        newrow[5:10] = sorted(newrow[5:10])
        newrow[10:15] = sorted(newrow[10:15])
        pbp_fin.append(newrow.copy())
    return pbp_fin

#LINEUP EVALUATION FUNCTIONS: RETURNS GAME PROGRESSION DIVIDED INTO LINEUP PARINGS

def game_by_lineup(pbp_fin):
    game_by_lineup = list()
    prev_scorediff = 0
    secs = 0
    iteration = iter(range(1,len(pbp_fin)))

    for j in iteration:
        #print(j)
        if "entered" in pbp_fin[j][1] or "left" in pbp_fin[j][1] or "End of " in pbp_fin[j][1]:
            #print("ywa")
            #print(pbp_fin[j][1])
            time = ms_to_s(pbp_fin[j][0],10) - secs
            secs =  ms_to_s(pbp_fin[j][0],10)
        
            score = pbp_fin[j][4] - prev_scorediff
            prev_scorediff = pbp_fin[j][4]
        
            newrow = [time,score,pbp_fin[j-1][5:15]]
            game_by_lineup.append(newrow.copy())

            skip_num = find_next(pbp_fin,j)
            for iters in range(0,skip_num):
                next(iteration)
            #print(j)
    return game_by_lineup

def plus_minus(game_by_lineup,*players):
    sumz = 0
    dy_mins = 0
    for i in range(0,len(game_by_lineup)):
        for player in players:
            if all(player in string for string in game_by_lineup[i][2] == False):
                continue
            else:
                sumz = sumz + game_by_lineup[i][1]
                dy_mins = dy_mins + game_by_lineup[i][0]
        
    return [dy_mins,sumz]

def lp_pairings_summary(pbp_fin):
    gblp = game_by_lineup(pbp_fin)
    list_lps = list()
    for i in range(1,len(gblp)):
        #print(list_lps)
        if "" in gblp[i][2]:
            #print(gblp[i][0])
            index = find_ind(gblp[i-1][2],list_lps)
            list_lps[index][0] += gblp[i][0]
            list_lps[index][1] += gblp[i][1]  
        
        else:
            if find_ind(gblp[i][2],list_lps) == False:
                #print(gblp[i][0])
                list_lps.append([gblp[i][0],gblp[i][1],gblp[i][2][0:5],gblp[i][2][5:10]])
            else:
                #print(gblp[i][0])
                index = find_ind(gblp[i][2],list_lps)
                list_lps[index][0] += gblp[i][0]
                list_lps[index][1] += gblp[i][1]                 
    
    return list_lps

def lp_summary(pbp_fin,team):
    if team == "home":
        ind = 2
    else:
        ind = 3 
    list_lps = list()
    lp_paired = lp_pairings_summary(pbp_fin)
    for i in range(0,len(lp_paired)):
        if find_ind(lp_paired[i][ind],list_lps) == False:
            list_lps.append([lp_paired[i][0],lp_paired[i][1],lp_paired[i][ind]])
        else:
            index = find_ind(lp_paired[i][2],list_lps)
            list_lps[index][0] += lp_paired[i][0]
            list_lps[index][1] += lp_paired[i][1]  
    return list_lps
            
def find_ind(qlist,lp_list):
    for i in range(len(lp_list)):
        if qlist == lp_list[i][2]:
            return i
    return False

In [6]:
#HELPER FUNCTIONS

def ms_to_s(string,lengthofq):
    try:
        mins = int(string[0:2])
        secs = int(string[3:])
        secs = 60*mins + secs
    except:
        if "End of quarter 1" in string:
            return 60*lengthofq
        elif "End of quarter 2" in string:
            return 60*2*lengthofq
        elif "End of quarter 3" in string:
            return 60*3*lengthofq
        elif "End of game" in string or "End of quarter 4" in string:
            return 60*4*lengthofq
    return secs


def find_next(pbp,index):
    if "End of quarter 4" in pbp[index][1] or "End of game" in pbp[index][1]:
        return 0
    h = 1
    while True:
        if "End of quarter 4" in pbp[index+h][1]:
            return h
        if "entered" not in pbp[index+h][1] and "left" not in pbp[index+h][1] and "End of " not in pbp[index+h][1]:
            return h
        h = h + 1

In [52]:
#Example raw_scrape for single given link

raw_scrape("https://www.basket.fi/basketball-finland/competitions/game/?game_id=5267011&season_id=119177&league_id=1")

[['Kouvottaret', 'Tapiolan Honka\nLegend', '', ''],
 ['00:00', 'Kouvottaret', '(24) Aliyah Collier entered the court', ''],
 ['00:00', 'Kouvottaret', '(5) Sarah Toeaina entered the court', ''],
 ['00:00', 'Tapiolan Honka', '(8) Cierra Dillard entered the court', ''],
 ['00:00', 'Kouvottaret', '(4) Henriikka Vahauski entered the court', ''],
 ['00:00', 'Tapiolan Honka', '(5) Taru Tuukkanen entered the court', ''],
 ['00:00', 'Kouvottaret', '(15) Roosa Frimodig entered the court', ''],
 ['00:00', 'Tapiolan Honka', '(24) Gabriela Bronson entered the court', ''],
 ['00:00', 'Kouvottaret', '(41) Kaisa Kuisma entered the court', ''],
 ['00:00', 'Tapiolan Honka', '(13) Pipsa Kahlos entered the court', ''],
 ['00:00',
  'Tapiolan Honka',
  '(45) Lotta-Kaisa Paltemaa entered the court',
  ''],
 ['Start of game', 'Tapiolan Honka', 'Start of game', ''],
 ['Start of quarter 1', 'Tapiolan Honka', 'Start of quarter 1', ''],
 ['00:06',
  'Tapiolan Honka',
  'Tapiolan Honka\n(13) Pipsa Kahlos missed a

In [ ]:
#Example: Finding 2021 Play-by-Plays
for i in range(0,len(fin_w_pbps_2021)):
    if (i == 0) or (i % 2 == 0):
        continue
    else:
        print(i)
        fin_w_pbps_2021[i] = raw_v1_topbp(fin_w_pbps_2021[i])

In [ ]:
#AFTER OPENING DOC

import dbm
import pickle
db = dbm.open('Bball_Finland','c')
pchdr = db["fin_w_results"]
db.close()
fin_w_results = pickle.loads(pchdr)

db = dbm.open('Bball_Finland','c')
pchdr = db["fin_w_pbps_2021"]
db.close()
fin_w_pbps_2021 = pickle.loads(pchdr)

In [ ]:
#BEFORE CLOSING DOC ALWAYS

import dbm
import pickle
db = dbm.open('Bball_Finland', 'c')

pchdr = pickle.dumps(fin_w_results)
db["fin_w_results"] = pchdr

db.close()


db = dbm.open('Bball_Finland', 'c')

pchdr = pickle.dumps(fin_w_pbps_2021)
db["fin_w_pbps_2021"] = pchdr

db.close()

In [ ]:
### FINDING GAMES AND CORRESPONDING LINKS FINNISH WOMEN's BASKETBALL LEAGUE


link = "https://www.basket.fi/basketball-finland/competitions/schedule-and-results/?league_id=1&season_id=119177#mbt:2-303$p&0=2:$t&0=1"
i = 1

driver.get(link)
fin_w_results = list()
links = list()
for a in driver.find_elements_by_xpath('.//a'):
    q_link = a.get_attribute('href')
    if "game_id" in q_link and q_link not in links:
        links.append(a.get_attribute('href'))


while True:
    try:
        date = driver.find_element_by_xpath('//*[@id="mbt-v2-schedule-table"]/tbody/tr['+str(i)+']/td[1]').text  
        home = driver.find_element_by_xpath('//*[@id="mbt-v2-schedule-table"]/tbody/tr['+str(i)+']/td[4]').text
        away = driver.find_element_by_xpath('//*[@id="mbt-v2-schedule-table"]/tbody/tr['+str(i)+']/td[6]').text
        link = links[i-1]
        
        fin_w_results.append([date,home,away,link])
        i = i+1
    except:
        break



